**Translation from other languages to english - data augmentation**

In [20]:
import csv
from deep_translator import GoogleTranslator
from tqdm import tqdm

In [21]:
# Load data spanish
DATA_PATH_SPA = '/Users/juliamf/Desktop/CMS-CLS/winter_semester24:25/LLMs/project/public_data_dev/track_a/train/esp.csv'

data_spanish = pd.read_csv(DATA_PATH_SPA)
data_spanish.head()

,id,text,anger,disgust,fear,joy,sadness,surprise
0,esp_train_track_a_00001,"2018 y lo sigo escuchando Like si tú también ,...",0,0,0,1,0,0
1,esp_train_track_a_00002,No les aguas. Caso a los pen...sativos de los ...,0,1,0,0,0,0
2,esp_train_track_a_00003,yo ya estoy.cagandome de risa pero jajajaaja e...,0,0,0,1,0,0
3,esp_train_track_a_00004,Donde le pongo donde son tontos y asquerosos y...,1,1,0,0,0,0
4,esp_train_track_a_00005,"El ritmo está bien, la letra es un asco y quié...",0,1,0,0,0,0


In [22]:
# Checking label distribution
labels = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']
label_counts = data_spanish[labels].sum()
print(label_counts)
total = label_counts.sum()
print("total labels:", total)

# Count how many labels each text has
label_combinations = data_spanish[labels].sum(axis=1)
print(label_combinations.value_counts())

anger       492
disgust     654
fear        317
joy         642
sadness     309
surprise    421
dtype: int64
total labels: 2835
1    1218
2     718
3      59
4       1
Name: count, dtype: int64


In [23]:
def translate(text, source_lang='es', target_lang='en'):
    translated_text = GoogleTranslator(source=source_lang, target=target_lang).translate(text)
    return translated_text

# Separate data based on the minority class 
anger_class = data_spanish[(data_spanish['anger'] == 1) & (data_spanish[labels].sum(axis=1) == 1)]

# Augment text for the minority class using back-translation
augmented_data = []
for text in tqdm(anger_class['text'], desc="Augmenting Anger Class", unit="sentence"):
    augmented_text = translate(text)
    augmented_data.append(augmented_text)
    
# Create new rows with augmented data
augmented_df = anger_class.copy()
augmented_df['text'] = augmented_data

# Check the distribution after augmentation
distribution_after_augmentation = augmented_df[labels].sum()

print("Distribution after augmentation:")
print(distribution_after_augmentation)

Augmenting Anger Class: 100%|██████████| 61/61 [01:13<00:00,  1.20s/sentence]

Distribution after augmentation:
anger       61
disgust      0
fear         0
joy          0
sadness      0
surprise     0
dtype: int64


In [24]:
augmented_df.head()

,id,text,anger,disgust,fear,joy,sadness,surprise
8,esp_train_track_a_00009,I hate them and I don't believe that,1,0,0,0,0,0
10,esp_train_track_a_00011,A total lack of respect to change the anthem t...,1,0,0,0,0,0
58,esp_train_track_a_00059,@<username> that the @<username> invited you t...,1,0,0,0,0,0
59,esp_train_track_a_00060,I love it. I love them.,1,0,0,0,0,0
60,esp_train_track_a_00061,Damn damn people like that if they want to sto...,1,0,0,0,0,0
